In [126]:
from pythae.models.normalizing_flows import MADE, MADEConfig

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
c = MADEConfig(input_dim=(10,), hidden_sizes=[20, 10, 3], output_dim=(10, ))#, context_dim=None)

In [121]:
made = MADE(c)

tensor(1)
tensor(1, dtype=torch.int32)
tensor(1, dtype=torch.int32)


In [122]:
x = torch.randn(3, 10)
h = torch.randn(3, 2)

In [123]:
made

MADE(
  (net): Sequential(
    (0): MaskedLinear(in_features=10, out_features=20, bias=True)
    (1): MaskedLinear(in_features=20, out_features=10, bias=True)
    (2): ReLU()
    (3): MaskedLinear(in_features=10, out_features=3, bias=True)
    (4): ReLU()
    (5): MaskedLinear(in_features=3, out_features=20, bias=True)
  )
)

In [125]:
made(x, h).mu.shape

torch.Size([3, 10])

In [85]:
masks = made._make_mask()

tensor(1)
tensor(1, dtype=torch.int32)
tensor(1, dtype=torch.int32)


In [86]:
masks

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1

In [87]:
def create_input_order(input_size, input_order="left-to-right"):
    """Returns a degree vectors for the input."""
    if input_order == "left-to-right":
        return np.arange(start=1, stop=input_size + 1)
    elif input_order == "right-to-left":
        return np.arange(start=input_size, stop=0, step=-1)
    elif input_order == "random":
        ret = np.arange(start=1, stop=input_size + 1)
        np.random.shuffle(ret)
        return ret


def create_degrees(
    input_size, hidden_units, input_order="left-to-right", hidden_degrees="equal"
):
    input_order = create_input_order(input_size, input_order)
    degrees = [input_order]
    for units in hidden_units:
        if hidden_degrees == "random":
            # samples from: [low, high)
            degrees.append(
                np.random.randint(
                    low=min(np.min(degrees[-1]), input_size - 1),
                    high=input_size,
                    size=units,
                )
            )
        elif hidden_degrees == "equal":
            min_degree = min(np.min(degrees[-1]), input_size - 1)
            degrees.append(
                np.maximum(
                    min_degree,
                    # Evenly divide the range `[1, input_size - 1]` in to `units + 1`
                    # segments, and pick the boundaries between the segments as degrees.
                    np.ceil(
                        np.arange(1, units + 1) * (input_size - 1) / float(units + 1)
                    ).astype(np.int32),
                )
            )
    return degrees


def create_masks(degrees):
    """Returns a list of binary mask matrices enforcing autoregressivity."""
    return [
        # Create input->hidden and hidden->hidden masks.
        (inp[:, np.newaxis] <= out).astype(np.int)
        for inp, out in zip(degrees[:-1], degrees[1:])
    ] + [
        # Create hidden->output mask.
        (degrees[-1][:, np.newaxis]
        < degrees[0]).astype(np.int)
    ]


In [88]:
deg = create_degrees(10, [20, 10, 3])
msks = create_masks(deg)

In [89]:
msks

[array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 

In [90]:
made._make_mask()

tensor(1)
tensor(1, dtype=torch.int32)
tensor(1, dtype=torch.int32)


[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1

In [91]:
for i in range(len(deg)):
    print(deg[i]- made.m[i-1].numpy())

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0]


In [92]:
made_masks = made._make_mask()

for i in range(len(deg)):
    print(abs(msks[i].T- made_masks[i].numpy()).sum())

tensor(1)
tensor(1, dtype=torch.int32)
tensor(1, dtype=torch.int32)
0.0
0.0
0.0
0.0


In [94]:
x = torch.rand(3, 10)
h = torch.randn(3, 2)

In [95]:
made(x, h)

TypeError: forward() got an unexpected keyword argument 'h'

In [48]:
a = torch.arange(1, 10)
idx = torch.randperm(len(a))
a[idx]

tensor([6, 8, 5, 9, 2, 4, 3, 7, 1])

In [44]:
a

tensor([1, 2, 3, 4, 3, 6, 3, 5, 9, 1])

In [15]:
import torch
import numpy as np
a = torch.randn(10)

In [17]:
torch.min(a)

tensor(-1.7427)